In [31]:
import sympy
from sympy import latex
from sympy.physics.mechanics import init_vprinting
from IPython.display import display, Math

init_vprinting()
def skew(v):
    return sympy.Matrix([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
C_ned_enu = sympy.Matrix([[0,1,0],[1,0,0],[0,0,-1]])

vn, ve, vd = sympy.symbols('v_n v_e v_d')
phi, lamb, h = sympy.symbols('\\phi \\lambda h')
w_ie = sympy.symbols('\\omega_{ie}')
C_b_n, f_ib_b, w_ib_b = sympy.symbols('C_b_n f_{ib}^b \omega_{ib}^b')
Re, Rn, r_es_e, g0, e = sympy.symbols('R_e R_n R_{eS}^e, g_0 e')

sL, cL, tL = sympy.sin(phi), sympy.cos(phi), sympy.tan(phi)
Re_h = Re + h
Rn_h = Rn + h
Re_h2 = Re_h * Re_h
Rn_h2 = Rn_h * Rn_h

In [32]:
# NED matrices
w_en_n = sympy.Matrix([ve / (Re + h), -vn / (Rn + h), ve * tL / (Re + h)])  # Groves 5.44
w_ie_n = sympy.Matrix([w_ie * cL, 0, -w_ie * sL])  # Groves 2.123

F11 = -skew(w_en_n + w_ie_n)
F12 = sympy.Matrix([[0, -1 / Re_h, 0], [1 / Rn_h, 0, 0], [0, tL / Re_h, 0]])
F13 = sympy.Matrix(
    [
        [w_ie * sL, 0, ve / Re_h2],
        [0, 0, -vn / Rn_h2],
        [w_ie * cL + ve / (Re_h * cL**2), 0, -ve * tL / Re_h2],
    ]
)
# F21 = -skew(C_b_n @ f_ib_b)
F22 = sympy.Matrix(
    [
        [vd / Rn_h, -2 * (w_ie * sL + ve * tL / Re_h), vn / Rn_h],
        [2 * w_ie * sL + ve * tL / Re_h, (vn * tL + vd) / Re_h, 2 * w_ie * cL + ve / Re_h],
        [-2 * vn / Rn_h, -2 * (w_ie * cL + ve / Re_h), 0],
    ]
)
F23 = sympy.Matrix(
    [
        [-ve**2 / (Re_h * cL**2) - 2*ve*w_ie*cL, 0, -vn * vd / Rn_h2 + ve**2 * tL / Re_h2],
        [vn * ve / (Re_h * cL**2) + 2 * w_ie * (vn * cL - vd * sL), 0, -ve * (vn * tL + vd) / Re_h2],
        [2 * ve * w_ie * sL, 0, ve**2 / Re_h2 + vn**2 / Rn_h2 - 2 * g0 / r_es_e],
    ]
)
F32 = sympy.Matrix([[1 / Rn_h, 0, 0], [0, 1 / (Re_h * cL), 0], [0, 0, -1]])
F33 = sympy.Matrix([[0, 0, -vn / Rn_h2], [ve * sL / (Re_h * cL**2), 0, -ve / (Re_h2 * cL)], [0, 0, 0]])
T_r_p = F32

print("NED")
display(Math( "F_{11} = " + latex(F11) ))
display(Math( "F_{12} = " + latex(F12) ))
display(Math( "F_{13} = " + latex(F13) ))
display(Math( "F_{22} = " + latex(F22) ))
display(Math( "F_{23} = " + latex(F23) ))
display(Math( "F_{32} = " + latex(F32) ))
display(Math( "F_{33} = " + latex(F33) ))

NED


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
# ENU Matrices
vu = sympy.symbols('v_u')
print("ENU")
# display(Math( "F_{11} = " + latex(-skew(C_ned_enu @ (w_en_n + w_ie_n))) ))
display(Math( "F_{11} = " + latex(C_ned_enu @ F11.subs(vd, vu)) ))
display(Math( "F_{12} = " + latex(C_ned_enu @ F12.subs(vd, vu)) ))
display(Math( "F_{13} = " + latex(C_ned_enu @ F13.subs(vd, vu)) ))
display(Math( "F_{22} = " + latex(C_ned_enu @ F22.subs(vd, vu)) ))
display(Math( "F_{23} = " + latex(C_ned_enu @ F23.subs(vd, vu)) ))
display(Math( "F_{32} = " + latex(C_ned_enu @ F32.subs(vd, vu)) ))
display(Math( "F_{33} = " + latex(C_ned_enu @ F33.subs(vd, vu)) ))

ENU


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
import navtools as nt
import numpy as np

R2D = 180.0 / np.pi
LLA_R2D = np.array([R2D, R2D, 1.0])
lla = np.array([32.2, -85.5, 200.0]) / LLA_R2D
ecef = nt.lla2ecef(lla)
sv_ecef = np.array([13392805.9056497, -17121749.1891207, 14687559.3031646])
az,el,r = nt.ecef2aer(sv_ecef, ecef)
print(f"True Az = {az * R2D}, El = {el * R2D}")
u_true = np.array([np.sin(az) * np.cos(el), np.cos(az) * np.cos(el), np.sin(el)])
# u = nt.ecef2nedDcm(lla) @ (-(ecef - sv_ecef) / r)
u = (-(ecef - sv_ecef) / r) @ nt.ned2ecefDcm(lla)
az = np.arctan2(u[1], u[0])
el = -np.arcsin(u[2])
print(f"Rotated Az = {az * R2D}, El = {el * R2D}")

True Az = 76.90991995950667, El = 53.70643414041454
Rotated Az = 76.90991995950664, El = 53.706434140414515


In [35]:
# C_e_n = sympy.Matrix(
#     [
#         [-sympy.sin(lamb), sympy.cos(lamb), 0.0],
#         [-sympy.sin(phi) * sympy.cos(lamb), -sympy.sin(phi) * sympy.sin(lamb), sympy.cos(phi)],
#         [sympy.cos(phi) * sympy.cos(lamb), sympy.cos(phi) * sympy.sin(lamb), sympy.sin(phi)],
#     ],
# )
C_e_n = np.array(
    [
        [-sympy.sin(phi) * sympy.cos(lamb), -sympy.sin(phi) * sympy.sin(lamb), sympy.cos(phi)],
        [-sympy.sin(lamb), sympy.cos(lamb), 0.0],
        [-sympy.cos(phi) * sympy.cos(lamb), -sympy.cos(phi) * sympy.sin(lamb), -sympy.sin(phi)],
    ],
)
T_r_p = sympy.Matrix(
    [
        [0, (Re + h) * sympy.cos(phi) * sympy.cos(lamb), 0],
        [(Re + h) * sympy.cos(phi) * sympy.sin(lamb), 0, 0],
        [0, 0, ((1-e**2)*Re + h)*sympy.sin(phi)],
    ]
)

display(Math(f"T_r^p = {sympy.latex(sympy.inv_quick(T_r_p) @ -C_e_n.T)}"))

<IPython.core.display.Math object>

In [36]:
Cy, Cp, Cr, Sy, Sp, Sr = sympy.symbols('C_y C_p C_r S_y S_p S_r')

Ry = 1 * sympy.Matrix([[Cy, -Sy, 0],[Sy, Cy, 0],[0, 0, 1]])
Rp = 1 * sympy.Matrix([[Cp, 0, Sp],[0, 1, 0],[-Sp, 0, Cp]])
Rr = 1 * sympy.Matrix([[1, 0, 0],[0, Cr, -Sr],[0, Sr, Cr]])
display(Math(f"C-b-ned = {sympy.latex((Ry @ Rp @ Rr))}")) # C.106
# display(Math(f"C-ned-b = {sympy.latex((Ry @ Rp @ Rr).T)}"))
# display(Math(f"C-b-enu = {sympy.latex(C_ned_enu @ (Ry @ Rp @ Rr))}")) # C.106

Ry = 1 * sympy.Matrix([[Cy, -Sy, 0],[Sy, Cy, 0],[0, 0, 1]])
Rr = 1 * sympy.Matrix([[Cr, 0, Sr],[0, 1, 0],[-Sr, 0, Cr]])
Rp = 1 * sympy.Matrix([[1, 0, 0],[0, Cp, -Sp],[0, Sp, Cp]])
display(Math(f"C-b-ned = {sympy.latex((Rr @ Rp @ Ry))}")) # C.106

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [37]:

R = sympy.Matrix([[0,1,0],[1,0,0],[0,0,-1]])
print( np.sqrt( 1 + np.trace(np.array([[0,1,0],[1,0,0],[0,0,-1]])) ) )

r, p, y, a,b,c,d = sympy.symbols('r p y a b c d')
Cr = sympy.cos(r/2)
Cp = sympy.cos(p/2)
Cy = sympy.cos(y/2)
Sr = sympy.sin(r/2)
Sp = sympy.sin(p/2)
Sy = sympy.sin(y/2)

# body-to-nav quaternion
q = sympy.Matrix(
    [
        Cy * Cp * Cr + Sy * Sp * Sr,
        Cy * Cp * Sr - Sy * Sp * Cr,
        Cy * Sp * Cr + Sy * Cp * Sr,
        Sy * Cp * Cr - Cy * Sp * Sr,
    ],
)
p = sympy.Matrix([0, sympy.cos(sympy.pi/4), sympy.cos(sympy.pi/4), 0])
r = sympy.Matrix(
    [
        (p[0] * q[0] - p[1] * q[1] - p[2] * q[2] - p[3] * q[3]),
        (p[0] * q[1] + p[1] * q[0] + p[2] * q[3] - p[3] * q[2]),
        (p[0] * q[2] - p[1] * q[3] + p[2] * q[0] + p[3] * q[1]),
        (p[0] * q[3] + p[1] * q[2] - p[2] * q[1] + p[3] * q[0]),
    ]
)
# r = sympy.collect(r, Cy, evaluate=False)
display(Math(f"p = {sympy.latex(p)}"))
display(Math(f"q = {sympy.latex(q)}"))
display(Math(f"r = {sympy.latex(r)}"))
display(Math(f"r0 = {sympy.latex(r[0].subs(Sp*Cr, a).subs(Cp*Sr, b).simplify())}"))
display(Math(f"r1 = {sympy.latex(r[1].subs(Sp*Sr, c).subs(Cp*Cr, d).simplify())}"))
display(Math(f"r2 = {sympy.latex(r[2].subs(Sp*Sr, c).subs(Cp*Cr, d).simplify())}"))
display(Math(f"r3 = {sympy.latex(r[3].subs(Sp*Cr, a).subs(Cp*Sr, b).simplify())}"))



0.0


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [38]:
import navtools as nt
import numpy as np

R2D = 180 / np.pi

rpy = np.array([-20,-2,240])
C_b_ned = nt.euler2dcm(rpy / R2D, "ned")
C_b_enu = nt.euler2dcm(rpy / R2D, "enu")
q_b_ned = nt.euler2quat(rpy / R2D, "ned")
q_b_enu = nt.euler2quat(rpy / R2D, "enu")
print(f"C_b_ned = \n{C_b_ned}")
print(f"C_b_enu = \n{C_b_enu}\n")
print(f"q_ned = {q_b_ned}")
print(f"q_enu = {q_b_enu}\n")
print(f"q_ned2 = {nt.dcm2quat(C_b_ned)}")
print(f"q_enu2 = {nt.dcm2quat(C_b_enu)}\n")

print(f"rpy_ned_dcm = {nt.dcm2euler(C_b_ned, 'ned') * R2D}")
print(f"rpy_enu_dcm = {nt.dcm2euler(C_b_enu, 'enu') * R2D}\n")
print(f"rpy_ned_quat = {nt.quat2euler(q_b_ned, 'ned') * R2D}")
print(f"rpy_enu_quat = {nt.quat2euler(q_b_enu, 'enu') * R2D}\n")

C_b_ned = 
[[-0.49969541  0.80782952  0.31259553]
 [-0.86549784 -0.48018348 -0.14260894]
 [ 0.0348995  -0.34181179  0.93912019]]
C_b_enu = 
[[-0.86549784 -0.48018348 -0.14260894]
 [-0.49969541  0.80782952  0.31259553]
 [-0.0348995   0.34181179 -0.93912019]]

q_ned = [-0.48970432  0.10169547 -0.1417672   0.85425393]
q_enu = [ 0.02833499  0.2577755  -0.95032199 -0.1721541 ]

q_ned2 = [ 0.48970432 -0.10169547  0.1417672  -0.85425393]
q_enu2 = [ 0.02833499  0.2577755  -0.95032199 -0.1721541 ]

rpy_ned_dcm = [ -20.   -2. -120.]
rpy_enu_dcm = [ -20.   -2. -120.]

rpy_ned_quat = [ -20.   -2. -120.]
rpy_enu_quat = [ -20.   -2. -120.]



In [39]:
# pi, lamb, phi_gnss, phi_ins = sympy.symbols('\pi \lambda \phi_{gnss} \phi_{ins}')
# psi, theta, r, p, y = sympy.symbols('\psi \\theta r p y')

# Ry = 1 * sympy.Matrix([[sympy.cos(y), -sympy.sin(y), 0],[sympy.sin(y), sympy.cos(y), 0],[0, 0, 1]])
# Rp = 1 * sympy.Matrix([[sympy.cos(p), 0, sympy.sin(p)],[0, 1, 0],[-sympy.sin(p), 0, sympy.cos(p)]])
# Rr = 1 * sympy.Matrix([[1, 0, 0],[0, sympy.cos(r), -sympy.sin(r)],[0, sympy.sin(r), sympy.cos(r)]])
# Q = (C_ned_enu @ (Ry @ Rp @ Rr)).T
# Z = 0.5 * sympy.Matrix([[0, 0, 0], [lamb, 0, 0], [0, -lamb, 0], [lamb, -lamb, 0]])
# e_enu = sympy.Matrix([sympy.sin(psi)*sympy.cos(theta), sympy.cos(psi)*sympy.cos(theta), sympy.sin(theta)])

# phi = 2.0*pi/lamb * Z @ Q @ e_enu
# d_phi_d_Q = sympy.diff(phi_gnss, r)
# # d_phi_d_e = sympy.diff(phi_gnss, e_enu)

# display(Math(f"{sympy.latex(phi_gnss)} = {sympy.latex(phi.subs({Q: 'Q', Z: 'z', e_enu: 'e_{enu}'}))}"))
# display(Math(f"h({sympy.latex(phi_gnss)}|{sympy.latex(Q)}) = {sympy.latex(d_phi_d_Q)}"))

def taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    function_expression:    Sympy expression of the function
    variable_list:          list. All variables to be approximated (to be "Taylorized")
    evaluation_point:       list. Coordinates, where the function will be expressed
    degree:                 int. Total degree of the Taylor polynomial
    
    Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, 
    approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

pi, lamb, Z, Q, e_enu, del_e, del_Q = sympy.symbols('\pi \lambda Z Q e_enu \delta_e \delta_Q')
phi_gnss, phi_ins = sympy.symbols('\phi_{gnss} \phi_{ins}')
phi = (2*pi/lamb) * (Z * Q * e_enu)
dh_dy = sympy.simplify(taylor_polynomial_sympy(phi, [Q, e_enu], [del_Q, del_e], 1))
dQde = del_Q*del_e

display(Math(f"{sympy.latex(phi_gnss)} = {sympy.latex(phi)}"))
display(Math(sympy.latex(dh_dy.subs({dQde: 0}))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [40]:
# dt, S_x, S_y, S_z, S_b, S_d = sympy.symbols('dt S_x S_y S_z S_b S_d')

# I3 = sympy.Identity(3)
# I8 = sympy.Identity(8)
# Z23 = sympy.Matrix([[0,0,0],[0,0,0]])
# Z32 = Z23.T
# Z3 = sympy.Matrix([[0,0,0],[0,0,0],[0,0,0]])

# clk_f = sympy.Matrix([[0, 1], [0, 0]])
# clk_qc = sympy.Matrix([[S_b, 0], [0, S_d]])
# Ac = sympy.Matrix(sympy.BlockMatrix(
#     [
#         [F33, F32, Z32],
#         [F23, F22, Z32],
#         [Z23, Z23, clk_f],
#     ]
# ))
# Sa = sympy.Matrix([[S_x,0,0],[0,S_y,0],[0,0,S_z]])
# Qc = sympy.Matrix(sympy.BlockMatrix([[Z3, Z3, Z32], [Z3, Sa, Z32], [Z23, Z23, clk_qc]]))
# Q_tmp = sympy.simplify(sympy.Matrix((I8 + Ac*dt) @ Qc @ (I8 + Ac.T*dt)))
# # Qd = sympy.integrate(Q_tmp, dt)

# display(Math( "Q_c = " + latex(Qc) ))
# display(Math( "A_tmp = " + latex(Q_tmp) ))